In [1]:
import re
import pandas as pd
import geopandas

In [2]:
# county_csv = pd.read_csv('src/data/mn_demographics_history.csv')
# county_csv.columns

In [3]:
# Bring in TIGER shapefile for data
county_tiger = geopandas.read_file('/Users/Coreymj/Documents/GIS/usa/tl_2019_us_county/tl_2019_us_county.shp')

In [4]:
county_tiger_ia = county_tiger[county_tiger['STATEFP'] == '19'].copy()
county_tiger_ia.GEOID = county_tiger_ia.GEOID.astype('int64')
county_tiger_ia.sort_values(['NAME']).head()

,STATEFP,COUNTYFP,COUNTYNS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CSAFP,CBSAFP,METDIVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
684,19,001,00465190,19001,Adair,Adair County,06,H1,G4020,None,None,None,A,1474404191,2597996,+41.3285283,-094.4781643,"POLYGON ((-94.70070 41.41758, -94.70067 41.427..."
2603,19,003,00465191,19003,Adams,Adams County,06,H1,G4020,None,None,None,A,1096686238,5367875,+41.0216555,-094.6969059,"POLYGON ((-94.92768 41.07232, -94.92767 41.074..."
3148,19,005,00465192,19005,Allamakee,Allamakee County,06,H1,G4020,None,None,None,A,1655115644,51094070,+43.2749637,-091.3827510,"POLYGON ((-91.61091 43.42833, -91.61091 43.431..."
1247,19,007,00465193,19007,Appanoose,Appanoose County,06,H1,G4020,None,None,None,A,1287978861,49088024,+40.7442964,-092.8730610,"POLYGON ((-92.86858 40.89824, -92.86630 40.898..."
1916,19,009,00465194,19009,Audubon,Audubon County,06,H1,G4020,None,None,None,A,1147264566,1152260,+41.6791780,-094.9043119,"POLYGON ((-95.09330 41.77694, -95.09329 41.777..."


In [5]:
county_tiger_ia.rename(
    columns={
        'STATEFP': 'statefp',
        'NAME': 'county',
        'GEOID': 'fips'
    }, inplace=True
)

In [6]:
county_out = county_tiger_ia.copy()[[
    'statefp',
    'county',
    'fips',
]]

county_out.fips = county_out.fips.astype('int64')
county_out.head()

,statefp,county,fips
134,19,Decatur,19053
175,19,Cedar,19031
185,19,Carroll,19027
189,19,Pocahontas,19151
259,19,Kossuth,19109


In [7]:
ia_incomes = pd.read_csv('src/data/acs2018_5yr_B19013_05000US19049.csv')
ia_incomes['geoid_split'] = ia_incomes['geoid'].str.split("US").str[1]
ia_incomes.geoid_split = ia_incomes.geoid_split.astype('int64')
ia_incomes.rename(columns={'B19013001': 'median_income'}, inplace=True)
ia_incomes.head()

,geoid,name,median_income,"B19013001, Error",geoid_split
0,04000US19,Iowa,58580,313,19
1,05000US19001,"Adair County, IA",51859,2747,19001
2,05000US19003,"Adams County, IA",49229,4027,19003
3,05000US19005,"Allamakee County, IA",51057,4814,19005
4,05000US19007,"Appanoose County, IA",41111,2506,19007


In [8]:
county_out = county_out.merge(
    ia_incomes[[
        'geoid_split',
        'median_income'
    ]],
    how="left",
    left_on="fips",
    right_on="geoid_split"
)
county_out.head(10)

,statefp,county,fips,geoid_split,median_income
0,19,Decatur,19053,19053,42614
1,19,Cedar,19031,19031,65791
2,19,Carroll,19027,19027,57674
3,19,Pocahontas,19151,19151,49877
4,19,Kossuth,19109,19109,54146
5,19,Harrison,19085,19085,60063
6,19,Webster,19187,19187,44261
7,19,Jefferson,19101,19101,43378
8,19,Madison,19121,19121,64512
9,19,Howard,19089,19089,52742


In [9]:
ia_age = pd.read_csv('src/data/acs2018_5yr_B01002_05000US19049.csv')
ia_age['geoid_split'] = ia_age['geoid'].str.split("US").str[1]
ia_age.geoid_split = ia_age.geoid_split.astype('int64')
ia_age.rename(columns={'B01002001': 'median_age'}, inplace=True)
ia_age.head()

,geoid,name,B01002000.5,"B01002000.5, Error",median_age,"B01002001, Error",B01002002,"B01002002, Error",B01002003,"B01002003, Error",geoid_split
0,04000US19,Iowa,NaN,NaN,38.1,0.1,36.9,0.1,39.3,0.2,19
1,05000US19001,"Adair County, IA",NaN,NaN,45.4,0.4,44.4,0.5,47.8,0.8,19001
2,05000US19003,"Adams County, IA",NaN,NaN,47.3,0.5,45.3,0.7,48.7,0.6,19003
3,05000US19005,"Allamakee County, IA",NaN,NaN,44.8,0.8,42.4,2.1,46.2,1.1,19005
4,05000US19007,"Appanoose County, IA",NaN,NaN,45.4,0.6,44.1,0.7,47.0,0.7,19007


In [10]:
county_out = county_out.merge(
    ia_age[[
        'geoid_split',
        'median_age'
    ]],
    how="left",
    left_on="fips",
    right_on="geoid_split"
)

In [11]:
ia_race = pd.read_csv('src/data/acs2018_5yr_B03002_05000US19049.csv')
ia_race['geoid_split'] = ia_race['geoid'].str.split("US").str[1]
ia_race.geoid_split = ia_race.geoid_split.astype('int64')
ia_race.rename(columns={'B03002001': 'total_pop', 'B03002003': 'not_hispanic_white_alone'}, inplace=True)
ia_race['pct_nonwhite'] = 1 - (ia_race.not_hispanic_white_alone / ia_race.total_pop)
ia_race.head()


,geoid,name,total_pop,"B03002001, Error",B03002002,"B03002002, Error",not_hispanic_white_alone,"B03002003, Error",B03002004,"B03002004, Error",...,B03002018,"B03002018, Error",B03002019,"B03002019, Error",B03002020,"B03002020, Error",B03002021,"B03002021, Error",geoid_split,pct_nonwhite
0,04000US19,Iowa,3132499,0,2949203,221,2695888,609,107471,1631,...,36362,1981,9302,1077,4497,629,4805,834,19,0.139381
1,05000US19001,"Adair County, IA",7124,0,6989,0,6867,13,27,13,...,11,12,15,13,10,11,5,7,19001,0.036075
2,05000US19003,"Adams County, IA",3726,0,3679,0,3643,34,16,32,...,28,13,0,9,0,9,0,9,19003,0.022276
3,05000US19005,"Allamakee County, IA",13880,0,12996,0,12602,15,174,66,...,17,20,29,37,29,37,0,15,19005,0.092075
4,05000US19007,"Appanoose County, IA",12510,0,12271,0,11938,15,74,46,...,10,12,11,23,11,23,0,15,19007,0.045723


In [12]:
county_out = county_out.merge(
    ia_race[[
        'geoid_split',
        'pct_nonwhite'
    ]],
    how="left",
    left_on="fips",
    right_on="geoid_split"
)

In [13]:
# Extract Iowa 2016 results from PDF text
ia_results_regex = r'([A-z \']+) Election Day.*?Total ([ \d,]+)'
ia_2016_results_txt = open('src/data/ia_2016_results_from_pdf.txt', 'r').read()
# Fix PottawattamieElection
ia_2016_results_txt = ia_2016_results_txt.replace('PottawattamieElection', 'Pottawattamie Election')
county_matches = re.findall(ia_results_regex, ia_2016_results_txt, re.DOTALL)
county_results_list = []
for c in county_matches[:99]:  # Get the first 99 rows
    county = {'county': c[0]}
    votes = c[1].split(' ')
    county['trump_votes'] = int(votes[0].replace(',', ''))
    county['clinton_votes'] = int(votes[1].replace(',', ''))
    county['total_votes'] = int(votes[13].replace(',', ''))
    county_results_list.append(county)
ia_2016_results = pd.DataFrame(county_results_list)

In [14]:
ia_2016_results['rPct_2016'] = ia_2016_results.trump_votes / ia_2016_results.total_votes
ia_2016_results['dPct_2016'] = ia_2016_results.clinton_votes / ia_2016_results.total_votes
ia_2016_results

,county,trump_votes,clinton_votes,total_votes,rPct_2016,dPct_2016
0,Adair,2461,1133,3848,0.639553,0.294439
1,Adams,1395,565,2131,0.654622,0.265134
2,Allamakee,4093,2421,7035,0.581805,0.344136
3,Appanoose,4033,1814,6196,0.650904,0.292770
4,Audubon,2136,1080,3486,0.612737,0.309811
...,...,...,...,...,...,...
94,Winnebago,3447,1931,5864,0.587824,0.329297
95,Winneshiek,5344,5254,11511,0.464252,0.456433
96,Woodbury,24727,16210,44167,0.559852,0.367016
97,Worth,2453,1530,4305,0.569803,0.355401


In [15]:
county_out = county_out.merge(
    ia_2016_results[[
        'county',
        'rPct_2016',
        'dPct_2016'
    ]],
    how="left",
    left_on="county",
    right_on="county"
)

In [16]:
county_out.to_json('src/data/ia_demographics_history.json', orient='records')
county_out.head(25)

,statefp,county,fips,geoid_split_x,median_income,geoid_split_y,median_age,geoid_split,pct_nonwhite,rPct_2016,dPct_2016
0,19,Decatur,19053,19053,42614,19053,36.4,19053,0.070612,0.606765,0.317389
1,19,Cedar,19031,19031,65791,19031,42.0,19031,0.043481,0.549787,0.373689
2,19,Carroll,19027,19027,57674,19027,41.7,19027,0.014796,0.621419,0.309773
3,19,Pocahontas,19151,19151,49877,19151,47.9,19151,0.067991,0.689638,0.245789
4,19,Kossuth,19109,19109,54146,19109,45.5,19109,0.048823,0.646648,0.290895
5,19,Harrison,19085,19085,60063,19085,44.1,19085,0.036414,0.649100,0.282177
6,19,Webster,19187,19187,44261,19187,39.1,19187,0.123323,0.571591,0.358381
7,19,Jefferson,19101,19101,43378,19101,43.0,19101,0.192842,0.452384,0.447797
8,19,Madison,19121,19121,64512,19121,40.1,19121,0.045060,0.615667,0.307604
9,19,Howard,19089,19089,52742,19089,42.2,19089,0.033571,0.562716,0.361422
